# Stand:
- next page button funktionier
- alle 100 einträge pro seite funktionieren
  
# To-DO:
1. code schipsel zusammenknüpfen, sodass über alle seiten iteriert wird
   1. evt. checken ob es auch für andere willhaben links funktioniert
2. daten extrahieren (am besten mit dem code was wir schon bei dir geschrieben haben) und in ein pandas data frame schreiben

In [12]:
import bs4 as bs
import pandas as pd
from selenium import webdriver
import re as re
import psycopg2 as pg
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options



In [2]:
# create browser and launch website
driver = webdriver.Chrome('C:/Users/thoma/chromedriver/chromedriver.exe')
driver.get('https://www.willhaben.at/iad/immobilien/eigentumswohnung/eigentumswohnung-angebote?sfId=30fdd1af-4da1-4099-a7d9-dd2f309dd6a7&isNavigation=true&page=1&rows=100')

# remove privacy notice
privacy_deny = driver.find_element('xpath','//*[@id="didomi-notice-disagree-button"]')

privacy_deny.click()

C:\Users\thoma\AppData\Local\Temp\ipykernel_12028\3960068653.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/thoma/chromedriver/chromedriver.exe')


In [14]:
s = Service(executable_path="C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
options = Options()
driver = webdriver.Chrome(service=s, options=options)

In [4]:
# scroll slowly through page
page_height = driver.execute_script("return document.documentElement.scrollHeight")
current_height = 0
while current_height < page_height:
    current_height += 250
    driver.execute_script("window.scrollTo(0," + str(current_height) + ")")

In [ ]:
//*[@id="skip-to-content"]/div/div[3]/div[2]/div[2]/div[3]/div/nav/ul/li[7]

//*[@id="skip-to-content"]/div/div[4]/div[2]/div[2]/div[3]/div/nav/ul/li[7]

In [32]:
# iterate through found elements
i = 0
elements = driver.find_elements('css selector','.Box-sc-wfmb7k-0.ResultListAdRowLayout___StyledBox-sc-1rmys2w-0.ginNzk.dZyCtF')
for elem in elements:
    i = i + 1
    elem_html = elem.get_attribute('innerHTML')

    elem_soup = bs.BeautifulSoup(elem_html, 'html.parser')
    print('\n')
    print(i)
    print(re.search(pat_room, str(elem_soup)))
    print(re.search(pat_footage, str(elem_soup)))
    try:
        feat = re.search(pat_features, str(elem_soup)).group(0)
        if '<!-- -->' in feat: feat = feat - '<!-- -->'
    except AttributeError:
        feat = None
    
    print(feat)
    try:
        print(elem_soup.find("div", class_ = "Text-sc-10o2fdq-0 brNqZP").text)
    except AttributeError:
        print(None)





1
<re.Match object; span=(2443, 2444), match='3'>
<re.Match object; span=(2231, 2233), match='61'>


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [10]:
# go to next page
next_page = driver.find_element('xpath','/html/body/div[2]/div[1]/div[2]/div[1]/div/div[1]/div[3]/div/div/div[4]/div[2]/div[2]/div[3]/div/nav/ul/li[7]')
next_page_html = next_page.get_attribute('innerHTML')

print(next_page_html)
next_page.click()

<a tabindex="0" aria-label="Weiter zur nächsten Seite" class="Button__ButtonContainer-sc-3uaafx-0 jBEXJe Pagination__PaginationButton-sc-zvrf30-1 fYvOIb" href="/iad/immobilien/eigentumswohnung/eigentumswohnung-angebote?sfId=30fdd1af-4da1-4099-a7d9-dd2f309dd6a7&amp;isNavigation=true&amp;rows=100&amp;page=2" type="button" data-testid="pagination-bottom-next-button"><svg xmlns="http://www.w3.org/2000/svg" width="1em" height="1em" viewBox="0 0 24 24" class="createSvgIcon__SvgIcon-sc-1vebdtk-0 hwyjDC" pointer-events="none"><path fill="currentColor" d="M7.7 2v20l10-10-10-10z"></path></svg></a>


In [29]:
pat_room = re.compile(r'\d+(?=<\/span><span class=\"Text-sc-10o2fdq-0 iyzLep\"> (<!-- -->)?Zimmer)')
pat_footage = re.compile(r'\d+(?=</span><span class=\"Text-sc-10o2fdq-0 iyzLep\"> (<!-- -->)?m²)')
pat_features = re.compile(r'(?<=<span class=\"Text-sc-10o2fdq-0 iyzLep\"> )(?:<!-- -->|)[a-zA-Z/\u00fc\u00f6\u00e4\u00df\u00dc\u00d6\u00c4]*(?=<\/span>)')

In [ ]:
# check if link is contained in button, if len = 0 -> no more pages
href_pattern = re.compile(r'href="')
print(len(re.findall(href_pattern, str(next_page_html))))